In [10]:
import os
import pandas as pd
from datascience import *

nba_game_logs_fp = '/Volumes/Seagate_19X/NBA_HUDL_data/game_logs/15-16'
all_log_fps = [os.path.join(nba_game_logs_fp, f) for f in os.listdir(nba_game_logs_fp)]

example_log = all_log_fps[0]
df = pd.read_csv(example_log, delimiter=';')
example_table = Table.from_df(df)

In [18]:
num_games = 0
for fp in all_log_fps:
    if fp.endswith('.csv'):
        # temp_df = pd.read_csv(fp, delimiter=';')
        # df = pd.concat([df, temp_df])
        num_games += 1
# df.head()
num_games

655

In [34]:
sum(df['action_name'] == '3+')

10816

In [2]:
import os
import pandas as pd
from datascience import *

dir_path = '/Volumes/Seagate_19X/Euroleague_data/Game_log_data'
csv_fps = []

for root, dir, files in os.walk(dir_path):
    for file in files:
        fp = os.path.join(root, file)
        if fp.endswith('.csv'):
            csv_fps.append(fp)
csv_fps.__len__()

148817

In [88]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

keys = "id	action_id	action_name	player_id	player_name	team_id	team_name	opponent_id	opponent_name	opponent_team_id	opponent_team_name	teammate_id	teammate_name	half	second	pos_x	pos_y	possession_id	possession_name	possession_team_id	possession_team_name	possession_number	possession_start_clear	possession_end_clear	playtype	hand	shot_type	drive	dribble_move	contesting	ts".split()

def process_file(fp):
    try:
        league = fp.split('/')[-2]
        temp_df = pd.read_csv(fp, delimiter=';', names=keys)
        return league, {
            '1-': sum(temp_df['action_name'] == '1-'),
            '1+': sum(temp_df['action_name'] == '1+'),
            '2-': sum(temp_df['action_name'] == '2-'),
            '2+': sum(temp_df['action_name'] == '2+'),
            '3-': sum(temp_df['action_name'] == '3-'),
            '3+': sum(temp_df['action_name'] == '3+')
        }
    except Exception as e:
        print(f"Error: could not read file at {fp}")
        return league, None

def update_log_stats(results):
    log_stats = {}
    for league, stats in results:
        if stats is not None:
            if league not in log_stats:
                log_stats[league] = {
                    '1-': 0, 
                    '1+': 0, 
                    '2-': 0, 
                    '2+': 0, 
                    '3-': 0, 
                    '3+': 0
                    }
            for key in stats:
                log_stats[league][key] += stats[key]
    return log_stats

# Main processing with concurrent execution
def main(csv_fps):
    results = []
    with ThreadPoolExecutor(max_workers=32) as executor:
        future_to_fp = {executor.submit(process_file, fp): fp for fp in csv_fps}
        for future in tqdm(as_completed(future_to_fp), total=len(csv_fps)):
            result = future.result()
            if result:
                results.append(result)

    log_stats = update_log_stats(results)

    # Save the log statistics to a JSON file
    with open('log_stats.json', 'w') as f:
        json.dump(log_stats, f)

    print("Completed processing and saved the results.")

main(csv_fps)

  1%|          | 1309/148817 [00:05<09:27, 259.97it/s]


In [87]:
import numpy as np

ex = '/Volumes/Seagate_19X/Euroleague_data/Game_log_data/16-17/2_Euroleague/645_BC Barcelona - Brose Bamberg.csv'

keys = "id	action_id	action_name	player_id	player_name	team_id	team_name	opponent_id	opponent_name	opponent_team_id	opponent_team_name	teammate_id	teammate_name	half	second	pos_x	pos_y	possession_id	possession_name	possession_team_id	possession_team_name	possession_number	possession_start_clear	possession_end_clear	playtype	hand	shot_type	drive	dribble_move	contesting	ts".split()
df = pd.read_csv(ex, delimiter=';', names=keys)
sum(df['action_name'] == '2-')

32